In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!ls "/content/drive/My Drive"

'Colab Notebooks'   LSTM_IMDB.ipynb   Untitled0.ipynb
'Colab Uploads'     Reviews.csv


In [3]:
#importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
data = pd.read_csv('/content/drive/My Drive/Reviews.csv')

In [0]:
data = data[:10000]

In [6]:
data.head(2)

Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW  delmartian                     1   
1   2  B00813GRG4  A1D87F6ZCVE5NK      dll pa                     0   

   HelpfulnessDenominator  Score        Time                Summary  \
0                       1      5  1303862400  Good Quality Dog Food   
1                       0      1  1346976000      Not as Advertised   

                                                Text  
0  I have bought several of the Vitality canned d...  
1  Product arrived labeled as Jumbo Salted Peanut...

In [0]:
data = data.drop_duplicates(subset = ['UserId','ProfileName','Time','Summary'])

In [0]:
data =  data.dropna()

In [0]:
data = data.sort_values(by = 'Time')

In [0]:
def classifier(x):
  if x>3:
    return 1
  return 0 

In [0]:
data.Score = data.Score.map(classifier)

In [12]:
data.head(2)

Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
1244  1245  B00002Z754  A29Z5PI9BW2PU3      Robbie                     7   
1243  1244  B00002Z754  A3B8RCEI0FXFI6   B G Chase                    10   

      HelpfulnessDenominator  Score       Time  \
1244                       7      1  961718400   
1243                      10      1  962236800   

                             Summary  \
1244                   Great Product   
1243  WOW Make your own 'slickers' !   

                                                   Text  
1244  This was a really good idea and the final prod...  
1243  I just received my shipment and could hardly w...

In [0]:
text_data = data['Summary']+' '+data['Summary']+' '+data['Summary'] +' '+ data['Text']

In [0]:
#cleaning the data
# importing the libraries needed
import re
import nltk
import string
from bs4 import BeautifulSoup

In [15]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stopword = nltk.corpus.stopwords.words('english')
lemm = nltk.stem.wordnet.WordNetLemmatizer()
stopword = set(stopword)
def cleaner(phrase):
    # decontraction
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)  # r means raw string
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    #removing words with numbers and special characters
    phrase = re.sub(r"\S*\d\S","", phrase).strip()
    phrase = re.sub(r"[^A-Za-z0-9]+"," ",phrase)
    return phrase

In [17]:
from tqdm import tqdm
cleaned_text = []
words = []
for sent in tqdm(text_data):
    sent = BeautifulSoup(sent,'lxml').get_text()
    sent = cleaner(sent)
    sent = ' '.join(e.lower() for e in sent.split() if e.lower() not in stopword)
    sent = ' '.join(lemm.lemmatize(e) for e in sent.split())
    cleaned_text.append(sent)

100%|██████████| 9510/9510 [00:08<00:00, 1064.07it/s]


In [0]:
from collections import Counter
vocab_with_freq = Counter(str(cleaned_text).split())

In [0]:
vocab = vocab_with_freq.most_common(10000)

In [20]:
cleaned_text[0].split()

['great',
 'product',
 'great',
 'product',
 'great',
 'product',
 'really',
 'good',
 'idea',
 'final',
 'product',
 'outstanding',
 'use',
 'decal',
 'car',
 'window',
 'everybody',
 'asks',
 'bought',
 'decal',
 'made',
 'two',
 'thumb']

In [0]:
top_words = []
for (word,freq) in vocab:
  top_words.append(word)

In [0]:
dic = dict(vocab)

In [23]:
dic.get('like')

4760

In [0]:
final_text_data = []
for i in range(len(cleaned_text)):
  new_text = []
  for wor in str(cleaned_text[i]).split():
    if wor in top_words:
      new_text.append(dic.get(wor))
    else:
      new_text.append(0)
  final_text_data.append(new_text)


In [25]:
len(final_text_data)

9510

In [0]:
max_length_of_text=  []
for lst in final_text_data:
  max_length_of_text.append(len(lst))

In [27]:
sorted(max_length_of_text,reverse=True)[0]

874

Maximum Length of text data is 874

In [28]:
data.shape

(9510, 10)

In [0]:
y = data['Score']

In [0]:
from sklearn.model_selection import train_test_split
xtr,xtest,ytr,ytest = train_test_split(final_text_data,y,test_size = 0.2)

In [31]:
# truncate and/or pad input sequences
max_review_length = 874
X_train = sequence.pad_sequences(xtr, maxlen=max_review_length)
X_test = sequence.pad_sequences(xtest, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(7608, 874)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [32]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(10000, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 874, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 874, 100)          53200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 453,701
Trainable params: 453,701
Non-trainable params: 0
_______________

In [33]:
model.fit(X_train, ytr, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
7608/7608 [==============================] - 447s 59ms/step - loss: 0.5441 - acc: 0.7869
Epoch 2/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.3555 - acc: 0.8456
Epoch 3/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.3251 - acc: 0.8647
Epoch 4/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.3158 - acc: 0.8695
Epoch 5/10
7608/7608 [==============================] - 447s 59ms/step - loss: 0.3202 - acc: 0.8663
Epoch 6/10
7608/7608 [==============================] - 447s 59ms/step - loss: 0.3129 - acc: 0.8704
Epoch 7/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.2954 - acc: 0.8810
Epoch 8/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.2832 - acc: 0.8871
Epoch 9/10
7608/7608 [==============================] - 448s 59ms/step - loss: 0.2739 - acc: 0.8921
Epoch 10/10
7608/7608 [=============================